<h1 align=center><font size = 6>Segmenting and Clustering Neighborhoods in Toronto Part 2 </font></h1>

## By Alex

## Problem 2

<font size = 3> This notebook is the second part in a three part project for segmenting and clustering some neighborhood data from Toronto Canada. </font>
    
<font size = 3>In this notebook, we are once again using the Wikipedia page: <href> https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M </href> but now we are going to get the latitude and longitude coordinate of the neighborhoods to help us utilize Foursquare location data </font>

In [4]:
# import necessary libraries for the notebook
! pip install bs4
from bs4 import BeautifulSoup
import requests 
import pandas as pd 
import numpy as np

In [5]:
Url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(Url).text

In [6]:
#utilize BeautifulSoup from bs4 to pull the data out 
#of the wiki page (in XML) and bring it into our notebook
Bsoup = BeautifulSoup(source, "html.parser")
table = Bsoup.find('table')

In [7]:
#This dataframe will consist of 3 columns: PostalCode, Borough, and Neighborhood
column_Names = ['Postalcode', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns = column_Names)

In [8]:
#Now we search for all postalcodes, boroughs, and neighborhoods!
for tr_cell in table.find_all('tr'): #table reader
    row_data=[]
    for td_cell in tr_cell.find_all('td'): #data reader
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
df = df[df.Borough != 'Not assigned']
# we the combine the neighborhoods with the same Postalcode
df1 = df.groupby(['Postalcode', 'Borough'], sort=False).agg(', '.join)
df1.reset_index(inplace = True)
# now replace name of neighborhoods with "not assigned" rows with names of Borough
df1['Neighborhood'] = np.where(df1['Neighborhood'] == 'Not assigned',df1['Borough'],
df1['Neighborhood'])
df1.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df1.shape

(103, 3)

<font size = 4> Now, to continue we will define a function that helps us get the latitude and longitude coordinates for these locations in Toronto </font>

In [11]:
# I found the geocoder package a bit unreliable, so I created my own function to assist
def get_geocode(postal_code):
    # initialize variables 
    lat_lng_coords = None 
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitutde

In [12]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df1, on='Postalcode')

In [15]:
geo_data = geo_merged[['Postalcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

In [16]:
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### We've reached the end of part 2! Please continue to part 2 to see the next implementation step.